In [7]:
import pandas as pd
import janitor

In [9]:
file_path = '../data/CredLens Data File - CredLens Data File.csv'
fc_data = pd.read_csv(file_path)

Column names 

In [13]:
column_names = fc_data.columns
column_names

Index(['firstname', 'middleinitial', 'lastname', 'birthdate', 'degree',
       'major1', 'majorid1', 'cognateid1', 'studentuid', 'expectedgradterm',
       'studentssn', 'email1', 'address1', 'address2', 'city', 'state',
       'zipcode', 'phone1', 'phone2', 'mobilephone', 'graduationdate',
       'cohort', 'gender', 'ethnicorigin', 'ishispanic', 'cipcode', 'campus',
       'costtype'],
      dtype='object')

Clean the names 

In [12]:
fc_data = fc_data.clean_names()

Clean SSN 

In [17]:
# Remove dashes from SSNs, keeping NaN values unchanged
fc_data["ssn"] = fc_data["studentssn"].str.replace("-", "", regex=True)

# Keep rows where 'ssn' is either NaN or matches exactly 9 digits
fc_data = fc_data[fc_data["ssn"].isna() | fc_data["ssn"].str.fullmatch(r"\d{9}")]